In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import solver

In [ ]:
def differential_expression(f, x):
    f_prime = solver.calcGradients(f, x, 1)
    return f_prime - x - 1

def bc_function(x):
    return 0.0

bc_xs = [0.0]
bc_functions = [bc_function]
bc_ps = [0]

loss_fn = solver.LossFunction(differential_expression, bc_xs, bc_functions, bc_ps)

x_min = 0.0
x_max = 1.0
x_num = 200
x = np.linspace(x_min, x_max, x_num)
domain = [[x_min, x_max]]
dataloader = solver.genDataLoader(domain, num_points=x_num)


